In [9]:
import pygplates
import sys
import glob, re
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
import points_spatial_tree

import polygon_processing as pp

import paleogeography as pg
reload(pg)

%matplotlib inline



basedir = '../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles/'

sampling = 1.0

rotation_model = pygplates.RotationModel(['%s/Global_EB_250-0Ma_GK07_Matthews++.rot' % basedir,
                                          '%s/Global_EB_410-250Ma_GK07_Matthews++.rot' % basedir])


#####################################
MODELDIR = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/'
#static_polygon_file = '%s/ContinentalPolygons/Matthews_etal_GPC_2016_ContinentalPolygons.gpmlz' % MODELDIR
GPML_List = ['%s/DynamicPolygons/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_TopologyBuildingBlocks.gpmlz' % MODELDIR]
#####################################
topology_features = pygplates.FeatureCollection()
for file in GPML_List:
    topology_feature = pygplates.FeatureCollection(file)
    topology_features.add(topology_feature)
    
    
# make a sorted list of the (midpoint) times for paleogeography polygons
tmp = glob.glob('%s/*/' % basedir)

time_list = []
for tm in tmp:
    time_list.append(float(re.findall(r'\d+Ma+',tm)[1][:-2]))

time_list.sort()



In [10]:
def subduction_proximity(target_point_geometry,shared_boundary_sections):

    # The minimum distance of the current feature (its geometries) to all subducting lines in resolved topologies.
    min_distance_to_all_subducting_lines = None
    nearest_shared_sub_segment = None

    # Iterate over the shared boundary sections of all resolved topologies.
    for shared_boundary_section in shared_boundary_sections:

        # Skip sections that are not subduction zones.
        # We're only interesting in closeness to subducting lines.
        if shared_boundary_section.get_feature().get_feature_type() != pygplates.FeatureType.gpml_subduction_zone:
            continue

        # Iterate over the shared sub-segments of the current subducting line.
        # These are the parts of the subducting line that actually contribute to topological boundaries.
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():

            # Get the minimum distance from the current reconstructed geometry to
            # the current subducting line.
            min_distance_to_subducting_line = pygplates.GeometryOnSphere.distance(
                    target_point_geometry,
                    shared_sub_segment.get_resolved_geometry(),
                    min_distance_to_all_subducting_lines)

            # If the current subducting line is nearer than all previous ones
            # then it's the nearest subducting line so far.
            if min_distance_to_subducting_line is not None:
                min_distance_to_all_subducting_lines = min_distance_to_subducting_line
                nearest_shared_sub_segment = shared_sub_segment
                
    return min_distance_to_all_subducting_lines


In [18]:

area_threshold = 0.001

pg_mode = 'mountains'

pg_features_complete_list = []

for time in time_list:
    
    env_list = ['m']

    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,time)

    pg_features = pg.load_paleogeography(pg_dir,env_list)
    #cf = pp.merge_polygons(pg_features,rotation_model,time=t1,sampling=0.25)
    #sieve_polygons_t1 = pp.polygon_area_threshold(cf,area_threshold)
    
    resolved_topologies = []
    shared_boundary_sections = []
    pygplates.resolve_topologies(topology_features, rotation_model, resolved_topologies, time, shared_boundary_sections)
    ##################################### 
    
    for polygon in pg_features:
        for geometry in polygon.get_geometries():
            ds = subduction_proximity(geometry,shared_boundary_sections)
            #print ds * pygplates.Earth.mean_radius_in_kms
            new_polygon = polygon.clone()
            new_polygon.set_geometry(geometry)
            new_polygon.set_shapefile_attribute('DistanceToSubduction',float(ds * pygplates.Earth.mean_radius_in_kms))
            pg_features_complete_list.append(new_polygon)
    #break
    
    pygplates.FeatureCollection(pg_features_complete_list).write('tmp.gpmlz')
    
    

['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_14Ma/m_fig62_20_11_PresentDay_Paleogeog_Matthews2016_14.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_22Ma/m_fig60_29_20_PresentDay_Paleogeog_Matthews2016_22.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_33Ma/m_fig58_37_29_PresentDay_Paleogeog_Matthews2016_33.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_45Ma/m_fig56_49_37_PresentDay_Paleogeog_Matthews2016_45.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_53Ma/m_fig54_58_49_PresentDay_Paleogeog_Matthews2016_53.